# Contents
* [load_data](#load_data)
* [evaluation](#evaluation)
* [train](#train)

In [1]:
import numpy as np
import os
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

from evaluation import *
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
use_gpu = torch.cuda.is_available()
use_gpu

True

### load_data

In [3]:
class MarketDataset(Dataset):
    def __init__(self, paths, labels=None, le=None, transform=None):
        self.transform = transform
        self.img_paths = paths
        
        if le is not None:
            self.labels = torch.from_numpy(le.transform(labels))
        else:
            self.labels = None

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img = Image.open(img_path)

        if self.transform:
            img = self.transform(img)

        if self.labels is not None:
            label = self.labels[index]
            return (img, label)
        else:
            return img
        
def create_test_df(x):
    df = pd.DataFrame({'path': x})
    df['label'] = df.path.apply(lambda x: int(x.split('/')[-1].split('_')[0]))
    df['camera'] = df.path.apply(lambda x: int(x.split('/')[-1].split('_')[1].split('s')[0].split('c')[1]))
    df['name'] = df.path.apply(lambda x: x.split('/')[-1])
    df['is_junk'] = df['label'] == -1
    df['is_distractor'] = df['label'] == 0
    
    return df

def create_train_df(x):
    df = pd.DataFrame({'path': x})
    df['label'] = df.path.apply(lambda x: int(x.split('/')[-1].split('_')[0]))

    return df

def create_df(x, is_train):
    if is_train:
        return create_train_df(x)
    else:
        return create_test_df(x)

In [4]:
MARKET_ROOT = 'data/Market-1501-v15.09.15'
paths = {
    'train': 'bounding_box_train/*.jpg',
    'test': 'bounding_box_test/*.jpg',
    'query': 'query/*.jpg',
}
is_train = {
    'train': True,
    'test': False,
    'query': False,
}

dfs = {
    x: create_df(sorted(glob(os.path.join(MARKET_ROOT, paths[x]))), is_train[x]) 
    for x in paths.keys()
}

for key in dfs.keys():
    print(key, len(dfs[key]))
    dfs[key].to_csv('%s.csv' % key, index=None, header=None)
    
train_cl = np.array([int(x.split('/')[-1].split('_')[0]) for x in dfs['train']['path']])
train_label_encoder = preprocessing.LabelEncoder()
train_label_encoder.fit(train_cl)
train_cl_n = train_label_encoder.classes_.shape[0]

# todo try equal data transformers for train and test
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale([256, 256]),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Scale([256, 256]),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

transform_dict = {
    'train': 'train',
    'test': 'test',
    'query': 'test'
}

image_datasets = {
    x[0]: MarketDataset(dfs[x[0]]['path'], dfs[x[0]]['label'], le=x[1], transform=data_transforms[transform_dict[x[0]]]) 
    for x in [('train', train_label_encoder), ('test', None), ('query', None)]
}
dataset_size = len(image_datasets['train'])
dataloders = {
    x[0]: torch.utils.data.DataLoader(image_datasets[x[0]], batch_size=x[1],
                                      shuffle=x[2], num_workers=4)
    for x in [('train', 64, True), ('test', 16, False), ('query', 16, False)]
}

train 12936
test 19732
query 3368


### evaluation

In [5]:
def get_descr(dataloder, model):
    result = []
    
    for data in dataloder:
        if use_gpu:
            inputs = Variable(data.cuda())
        else:
            inputs = Variable(data)

        out = model(inputs)
        result.extend(out.data.squeeze().cpu().numpy())
        
    return np.array(result)

def get_info(df):
    labels = np.array(df['label'])
    camera_labels = np.array(df['camera'])
    image_paths = np.array(df['path'])
    image_names = np.array(df['name'])
    
    return labels, camera_labels, image_paths, image_names

def ranks_map(model, dataloders, nr):
    gescr_gallery = get_descr(dataloders['test'], model)
    gescr_query = get_descr(dataloders['query'], model)

    ranks, img_names_sorted, places = ranking('cosine', gescr_query, query_image_names, gescr_gallery, test_image_names, nr, Index)
    ranks_w = ranks / len(gescr_query) * 1.0
    mAP_ = mAP(gescr_query, query_image_names, gescr_gallery, test_image_names, nr, Index)
    
    return ranks_w, mAP_

In [6]:
test_labels, test_camera_labels, test_image_paths, test_image_names = get_info(dfs['test'])
query_labels, query_camera_labels, query_image_paths, query_image_names = get_info(dfs['query'])
    
Index = dict()
Index['junk'] = set()
Index['distractor'] = set()
for name in test_image_names:
    if ifJunk(name):
        Index['junk'].add(name)
    elif ifDistractor(name):
        Index['distractor'].add(name)
for query in query_image_names:
    Index[query] = dict()
    Index[query]['pos'] = set()
    Index[query]['junk'] = set()

    person, camera = parse_market_1501_name(query)
    for name in test_image_names:
        if ifJunk(name) or ifDistractor(name):
            continue
        person_, camera_ = parse_market_1501_name(name)
        if person == person_ and camera != camera_ :
            Index[query]['pos'].add(name)

        elif person == person_ and camera == camera :
            Index[query]['junk'].add(name)

### train

In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    for epoch in range(num_epochs):
        print('Epoch %d/%d' % (epoch, num_epochs - 1))
        print('-' * 10)
        
        scheduler.step()
        model.train(True)

        running_loss = .0
        running_corrects = 0

        for data in dataloders['train']:
            inputs, labels = data

            if use_gpu:
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.data[0]
            running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects / dataset_size

        print('Loss: %.4f Acc: %.4f' % (epoch_loss, epoch_acc))

        model.train(False)
        
        if (epoch + 1) % 5 == 0:
            ranks, mAP = ranks_map(model, dataloders, 50)
            print('rank1: %.4f mAP: %.4f' % (ranks[1], mAP))
        
        if (epoch + 1) % 10 == 0:
            torch.save(model_conv, 'models/finetuned_resnet_market_cross_entropy_v%d.pt' % epoch)
        
    time_elapsed = time.time() - since
    print('Training complete in %.0fm %.0fs' % (
        time_elapsed // 60, time_elapsed % 60))

    return model

In [8]:
model_conv = models.resnet50(pretrained=True)
num_ftrs = model_conv.fc.in_features
model_conv.fc = torch.nn.Sequential()
model_conv.fc.add_module('dropout', torch.nn.Dropout(p=0.75))
model_conv.fc.add_module('fc1', nn.Linear(num_ftrs, train_cl_n))

if use_gpu:
    model_conv = model_conv.cuda()
    
model_conv

ResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): Bottleneck (
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (downsample): Sequential (
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (1): Bott

In [9]:
params_dict = dict(model_conv.named_parameters())
params = []
for key, value in params_dict.items():
    print(key)
    if ('conv1' in key) & (not 'layer' in key):
        params += [{'params':[value],'lr':0.00001}]
    elif ('conv' in key):
        params += [{'params':[value],'lr':0.002}]
    elif ('fc1' in key):
        params += [{'params':[value],'lr':0.1}]
    elif 'bias' in key:
        params += [{'params':[value], 'lr':0.0001, 'weight_decay':0}]
    else:
        params += [{'params':[value], 'lr':0.0001}]

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.conv3.weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.conv3.weight
layer1.1.bn3.weight
layer1.1.bn3.bias
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias
layer1.2.conv3.weight
layer1.2.bn3.weight
layer1.2.bn3.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.conv3.weight
layer2.0.bn3.weight
layer2.0.bn3.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.we

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(params, momentum=0.9, weight_decay=0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=40, gamma=0.1)

In [11]:
model_conv = train_model(model_conv, criterion, optimizer_ft,
                         exp_lr_scheduler, num_epochs=50)

Epoch 0/49
----------
Loss: 0.1009 Acc: 0.0528
Epoch 1/49
----------
Loss: 0.0626 Acc: 0.2301
Epoch 2/49
----------
Loss: 0.0412 Acc: 0.4230
Epoch 3/49
----------
Loss: 0.0296 Acc: 0.5528
Epoch 4/49
----------
Loss: 0.0227 Acc: 0.6427
rank1: 0.6102 mAP: 0.3942
Epoch 5/49
----------
Loss: 0.0176 Acc: 0.7092
Epoch 6/49
----------
Loss: 0.0142 Acc: 0.7659
Epoch 7/49
----------
Loss: 0.0113 Acc: 0.8059
Epoch 8/49
----------
Loss: 0.0103 Acc: 0.8256
Epoch 9/49
----------
Loss: 0.0076 Acc: 0.8648
rank1: 0.6740 mAP: 0.4502
Epoch 10/49
----------
Loss: 0.0064 Acc: 0.8860
Epoch 11/49
----------
Loss: 0.0052 Acc: 0.9030
Epoch 12/49
----------
Loss: 0.0047 Acc: 0.9154
Epoch 13/49
----------
Loss: 0.0041 Acc: 0.9258
Epoch 14/49
----------
Loss: 0.0037 Acc: 0.9331
rank1: 0.6850 mAP: 0.4582
Epoch 15/49
----------
Loss: 0.0033 Acc: 0.9398
Epoch 16/49
----------
Loss: 0.0027 Acc: 0.9486
Epoch 17/49
----------
Loss: 0.0025 Acc: 0.9542
Epoch 18/49
----------
Loss: 0.0020 Acc: 0.9604
Epoch 19/49
--------

In [ ]:
torch.save(model_conv, 'models/finetuned_resnet_market_cross_entropy.pt')